In [84]:
import requests
from bs4 import BeautifulSoup
import random 
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)
session = requests.session()
headers = {'User-Agent': ua.random}
from string import punctuation
punctuation = punctuation + '—–“”«»'
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from sklearn.metrics import accuracy_score
stopWords = stopwords.words('russian')
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def page(url, headers):
    posts_list = []
    response = session.get(url, headers=headers)
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    posts = soup.find_all('span', {'class': '_reachbanner_'})
    for post in posts:
        i = post.get_text()
        posts_list.append(i)
    return posts_list[:40]

In [86]:
def clean(posts):
    clean_rew = []
    for_train = []
    for_test = []
    for r in posts:
        r = r.replace('\n', '')
        r = r.replace('\r', '')
        r = r.lower()
        small_list = []
        for p in punctuation:
            if p in r:
                r = r.replace(p, ' ')
        w = nltk.word_tokenize(r)
        for elem in w:
            ana = morph.parse(elem) 
            n = ana[0].normal_form
            if n not in stopWords or n == 'не':
                small_list.append(n)
        clean_rew.append(small_list)
    for s in clean_rew[:30]:
        for c in s:
            for_train.append(c)
    for s in clean_rew[30:40]:
        for_test.append(s)
    return for_train, for_test

In [ ]:
def unique(first_list, sec_list):
    unique_list = []
    clean_unique_list = []
    for i in first_list:
        if i not in sec_list:
            unique_list.append(i)
    for c in Counter(unique_list).items():
        if c[1]>2:
            clean_unique_list.append(c[0])
    return clean_unique_list

In [ ]:
def simple_type_detect(words_list, bad_words, good_words):
    word_scores = {'negative':0, 'positive':0}
    for i in words_list:
        if i in bad_words:
            word_scores['negative'] += 1
        elif i in good_words:
            word_scores['positive'] += 1
    return Counter(word_scores).most_common(1)

In [ ]:
def accuracy(tests, first_uniques, second_uniques):
    gold = []
    result = []
    for t in tests.items():
        gold.append(t[1])
        n = simple_type_detect(t[0], first_uniques, second_uniques) 
        result.append(n[0][0])
    return accuracy_score(result, gold)

In [87]:
url_bad = 'https://www.kinopoisk.ru/film/462682/reviews/ord/rating/status/bad/perpage/100/'
url_good = 'https://www.kinopoisk.ru/film/462682/reviews/ord/rating/status/good/perpage/200/'

In [88]:
bad = clean(page(url_bad, headers))

In [89]:
good = clean(page(url_good, headers))

In [90]:
test_rews = {}
for b in bad[1]:
    test_rews[tuple(b)] = 'negative'
for g in good[1]:
    test_rews[tuple(g)] = 'positive'

In [91]:
bad_uniques = unique(bad[0], good[0])

In [92]:
good_uniques = unique(good[0], bad[0])

In [93]:
accuracy(test_rews, bad_uniques, good_uniques)

0.55

In [94]:
#среди плохих тестовых рецензий правильно определяется только 6/10
#среди хороших тестовых рецензий правильно определяется 10/10
#видимо так проиходит потому что больше уникальных слов у хороших рецензий

# Как улучшить?

1. Можно воспользоваться нейросетью и обучить ее на большем количестве текстов (плохих и хороших; пусть еще смотрит не слова, а словосочетания; определяет синонимы), брать не только один фильм, и вообще брать разные типы текстов
2. Можно получше убрать шум (слова с опечатками, имена героев если они остаются и тд)

## Улучшение

Лучше всех с русским текстом работал pymystem3, поэтому берем его

In [95]:
from pymystem3 import Mystem
mystem = Mystem()
!wget http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
!tar -xvf mystem-3.0-linux3.1-64bit.tar.gz
!cp mystem /root/.local/bin/mystem

--2020-10-18 20:01:50--  http://download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving download.cdn.yandex.net (download.cdn.yandex.net)... 5.45.205.243, 5.45.205.245, 5.45.205.242, ...
Connecting to download.cdn.yandex.net (download.cdn.yandex.net)|5.45.205.243|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: http://cache-mskstoredata10.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz [following]
--2020-10-18 20:01:51--  http://cache-mskstoredata10.cdn.yandex.net/download.cdn.yandex.net/mystem/mystem-3.0-linux3.1-64bit.tar.gz
Resolving cache-mskstoredata10.cdn.yandex.net (cache-mskstoredata10.cdn.yandex.net)... 37.9.96.21, 2a02:6b8:0:3706::19
Connecting to cache-mskstoredata10.cdn.yandex.net (cache-mskstoredata10.cdn.yandex.net)|37.9.96.21|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16457938 (16M) [application/octet-stream]
Saving to: ‘mystem-3.0-linux3.1-64bit.tar.gz.1’

mystem-

Синтаксические группы:
*   Местоимение + Глагол (для таких штук: Я люблю / Я ненавижу, просто отследить эмоции рецензента) 
*   Наречие + Прилагательное (для таких штук: Очень крутой / Немножко скучный, фильм может быть очень хорошим но немножко скучным, такое наверное встретится только в хорошем отзыве, в плохом написали бы, что он очень скучный)
*   Не + Глагол (для штук типа: Не люблю / Не напрягает, чтобы различать хорошее и плохое отношение человека к фильму)

Такие группы помогут взглянут на картину более целостно и точно


In [151]:
def pos(word_list):
  mystem_tags = []
  for i in word_list:
    m = mystem.analyze(i)
    try:
      gr = m[0]['analysis'][0]['gr']
      pos = gr.split('=')[0].split(',')[0]
    except:
      pos = ''
    mystem_tags.append(pos)
  return mystem_tags

In [152]:
def syntax_groups(words_list, pos_list):
  group_prontoverb = []
  group_advtoadj = []
  group_netoverb = []
  for t in range(len(pos_list)):
    if pos_list[t]=='SPRO' and pos_list[t+1]=='V':
      group_prontoverb.append(words_list[t] + ' ' + words_list[t+1])
    if pos_list[t]=='ADV' and pos_list[t+1]=='A':
      group_advtoadj.append(words_list[t] + ' ' + words_list[t+1])
    if words_list[t]=='не' and pos_list[t+1]=='V':
      group_netoverb.append(words_list[t] + ' ' + words_list[t+1])
  return group_prontoverb, group_advtoadj, group_netoverb

In [153]:
def groups_unique(first_list, sec_list):
    unique_list = []
    for i in first_list:
        if i not in sec_list:
            unique_list.append(i)
    return unique_list

In [154]:
def simple_type_detect_new(words_str, bad_words, good_words):
  word_scores = {'negative':0, 'positive':0}
  for i in bad_words:
    if i in words_str:
      word_scores['negative'] += 1
  for m in good_words:
    if m in words_str:
      print(m)
      word_scores['positive'] += 1
  return Counter(word_scores).most_common(1)

In [155]:
train_good_pos = pos(good[0])
train_bad_pos = pos(bad[0])

In [156]:
good_syntax_groups = syntax_groups(good[0], train_good_pos)
bad_syntax_groups = syntax_groups(bad[0], train_bad_pos)

In [157]:
unique_good_bigrams = set(groups_unique(good_syntax_groups[0], bad_syntax_groups[0]) + groups_unique(good_syntax_groups[1], bad_syntax_groups[1]) + 
                          groups_unique(good_syntax_groups[2], bad_syntax_groups[2]))

In [158]:
unique_bad_bigrams = set(groups_unique(bad_syntax_groups[0], good_syntax_groups[0]) + groups_unique(bad_syntax_groups[1], good_syntax_groups[1]) + 
                          groups_unique(bad_syntax_groups[2], good_syntax_groups[2]))

In [159]:
test_rews_new = {}
for b in bad[1]:
  n = ''
  for m in b:
    n += m + ' '
  test_rews_new[n] = 'negative'
for g in good[1]:
  n2 = ''
  for k in g:
    n2 += k + ' '
  test_rews_new[n2] = 'positive'   

In [160]:
accuracy(test_rews_new, unique_bad_bigrams, unique_good_bigrams)

0.5

Качество упало:(